# Implementation of the `scaredyFish` model in Julia
### First step
Let's first define a spatial landscape

In [3]:
## ---
## init
## ---
using Distributions, Distances, IterTools, BenchmarkTools


## -
## Define my landscape
## -

## number of rows and cols
nrows = 5
ncols = 10

## landscape is a binary matrix, where 1s are foraging patches (white tiles)
landscape = rand( Bernoulli( 0.5 ), nrows, ncols )

## indices for white tiles
global whiteTiles = getindex.( findall( y -> y == 1, landscape ), [ 1 2 ] ) ;


### Second step
Define a function that will loop through `times` and `agents` and do all the calculations

In [4]:
## ---
## Function to loop through time steps
## ---
function scaredyFish( ; agents, maxtime, walk, run, drift, strengthExcitation, strengthInhibition, sensory, threshold, trigger )
    
    ## -
    ## Prep simulation, define some empty arrays to fill in
    ## -

    ## empty 3D array to populate ( time x number of variables x agents )
    ## my columns are 
    ## | 1. time 
    ## | 2. agent id 
    ## | 3. x 
    ## | 4. y 
    ## | 5. state 0 (feed) or 1 (flee)
    ## | 6. in a white tile or not
    r = zeros( maxtime, 6, agents );

    ## populate first row of each matrix
    for agent in 1 : size( r, 3 )

        ## put a random number in the first row
        r[ 1, 1, agent ] = 1
        r[ 1, 2, agent ] = agent
        
        ## sample a white tile
        sampledTile = whiteTiles[ sample( 1 : size( whiteTiles )[ 1 ], 1 ), : ]
        
        ## put agent somewhere in this sampled white tile
        r[ 1, 3, agent ] = sampledTile[ 1 ] + Float64.( rand( Uniform( -0.49, 0.49 ) ) )
        r[ 1, 4, agent ] = sampledTile[ 2 ] + Float64.( rand( Uniform( -0.49, 0.49 ) ) )
        
        ## fill in state and location
        r[ 1, 5, agent ] = 0
        r[ 1, 6, agent ] = 1

    end
    
    
    ## ---
    ## Start workhorse
    ## ---
    
    ## for t in ___time___
    for t in 2 : maxtime

        ## for a in ___agents____
        for a in 1 : agents
            
            ## add time info to output table
            r[ t, 1, a ] = t

            ## add agent id to output table
            r[ t, 2, a ] = a
            
            ## -
            ## first thing we need to know is where the nearest white tile is
            ## -

            ## grab my current x and y before calculating
            currX = r[ t - 1, 3, a ]
            currY = r[ t - 1, 4, a ]

            ## round them to find the nearest tile regardless of color
            currPatchX = round.( Int, currX )
            currPatchY = round.( Int, currY )

            ## check that it is actually a tile, and you are not out of bounds
            if ( 0.5 < currPatchX < ( 0.5 + size( landscape, 1 ) ) ) && ( 0.5 < currPatchY < ( 0.5 + size( landscape, 2 ) ) )

                ## note that I am in bounds
                inBounds = 1

                ## if this is a white tile, we are already done
                if landscape[ currPatchX, currPatchY ] == 1

                    ## make this my home patch
                    homePatchX = currPatchX
                    homePatchY = currPatchY

                    ## record status in output ( 1 if I'm in a white tile, 0 if not )
                    r[ t, 6, a ] = 1

                ## otherwise, find the nearest white tile
                else

                    ## distances to white tiles
                    toWhiteTiles = colwise( Euclidean(), [ currX, currY ], transpose( whiteTiles ) )

                    ## which min
                    homePatchX = whiteTiles[ argmin( toWhiteTiles ), 1 ]
                    homePatchY = whiteTiles[ argmin( toWhiteTiles ), 2 ]

                    ## record status in output
                    r[ t, 6, a ] = 0

                end

            ## if I'm out of bounds...
            else

                ## note it
                inBounds = 0

                ## distances to white tiles
                toWhiteTiles = colwise( Euclidean(), [ currX, currY ], transpose( whiteTiles ) )

                ## which min
                homePatchX = whiteTiles[ argmin( toWhiteTiles ), 1 ]
                homePatchY = whiteTiles[ argmin( toWhiteTiles ), 2 ]

                ## record status in output
                r[ t, 6, a ] = 0

            end


            ## -
            ## Here starts the step updating section
            ## -

            ## if feeding and in bounds
            if r[ t - 1, 5, a ] == 0 && inBounds == 1

                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( 0, walk ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( 0, walk ) )

            end

            ## if feeding and out of bounds
            if r[ t - 1, 5, a ] == 0 && inBounds == 0

                ## bias my walk in this direction
                xBias = ( homePatchX - r[ t - 1, 3, a ] )
                yBias = ( homePatchY - r[ t - 1, 4, a ] )

                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( xBias * drift, walk ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( yBias * drift, walk ) )

            end

            ## if starting fleeing
            if r[ t - 1, 5, a ] == 1 && r[ t - 2, 5, a ] == 0

                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( 0, run ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( 0, run ) )

            end
            
            ## if continuing fleeing
            if r[ t - 1, 5, a ] == 1 && r[ t - 2, 5, a ] == 1

                ## bias my walk in this direction
                xBias = ( r[ t - 1, 3, a ] - r[ t - 2, 3, a ] )
                yBias = ( r[ t - 1, 4, a ] - r[ t - 2, 4, a ] )
                
                ## random walk steps
                r[ t, 3, a ] = r[ t - 1, 3, a ] + rand( Normal( xBias, 0.005 ) )
                r[ t, 4, a ] = r[ t - 1, 4, a ] + rand( Normal( yBias, 0.005 ) )

            end

        end

        
        ## -
        ## Here starts the information updating section
        ## -

        ## extract current coordinates
        C = hcat( r[ t, 3, : ], r[ t, 4, : ] )

        ## calculate distances between agents
        D = pairwise( Euclidean(), C; dims = 1 )

        ## threshold this matrix
        D[ D .> sensory ] .= 0

        ## convert to 1 / D ^ 2
        D[ D .> 0 ] = 1 ./ D[ D .> 0 ] .^ 2
        
        ##
        ## cycle through agents and calculate excitation and inhibition
        ##
        
        ## grab the states at time t - 1
        currStates = r[ t - 1, 5, : ]

        ## for a in ___agents____
        for a in 1 : agents
           
            ## excitation information
            excitation = strengthExcitation * sum( D[ a, findall( currStates .== 1 ) ] )
            
            ## inhibition information
            inhibition = strengthInhibition * sum( D[ a, findall( currStates .== 0 ) ] )
            
            ## decision making function
            decision = excitation - inhibition
            
            ## if we pass threshold
            if decision > 1
                
                ## update state
                r[ t, 5, a ] = 1
                
            end
            
            ## force the agent back to feed state if its been fleeing for 2 timesteps
            if r[ t - 1, 5, a ] == 1 && r[ t - 2, 5, a ] == 1
                
                ## update state
                r[ t, 5, a ] = 0
                
            end
            
        end
        
        ## -
        ## trigger false alarm
        ## -
        
        ## initiate
        if t == trigger

            ## switch state
            r[ t, 5, sample( 1 : agents, 1 ) ] .= 1

        end

    end
    
    ## return results
    return r
    
    end;


## ---
## a function to flatten a 3d array, R equivalent of do.call( rbind )
## ---
function flatten( ; data ) 
    
    s = data[ :, :, 1 ]
    
    ## loop through and append stack
    for i in 2 : size( r, 3 )
    
        ## add
        s = vcat( s, r[ :, :, i ] )
    
    end
    
    ## output
    return s
    
    end;


### Third step
Pass parameter values to our function and let it rip

In [5]:
## ---
## run my simulation once
## ---
r = scaredyFish( 
    agents = 100,
    maxtime = 50,
    walk = 0.1,
    run = 0.5,
    drift = 0.2,
    strengthExcitation = 5,
    strengthInhibition = 0.25,
    sensory = 2.5,
    threshold = 1,
    trigger = 5
    ) ;

### Fourth step
Flatten my array from 3D to 2D

In [6]:
## flatten
s = flatten( data = r ) ;

5000×6 Array{Float64,2}:
  1.0    1.0   4.71052    2.08723  0.0  1.0
  2.0    1.0   4.70565    1.90258  0.0  1.0
  3.0    1.0   4.78554    1.79858  0.0  1.0
  4.0    1.0   4.76315    1.79165  0.0  1.0
  5.0    1.0   4.66856    1.82179  0.0  1.0
  6.0    1.0   4.86118    1.87414  0.0  1.0
  7.0    1.0   4.88325    1.79114  0.0  1.0
  8.0    1.0   4.82201    1.71536  0.0  1.0
  9.0    1.0   4.85534    1.90616  0.0  1.0
 10.0    1.0   4.7745     1.96287  0.0  1.0
 11.0    1.0   4.61291    1.89985  0.0  1.0
 12.0    1.0   4.54023    1.79716  0.0  1.0
 13.0    1.0   4.46604    1.76778  0.0  1.0
  ⋮                                     ⋮
 39.0  100.0   2.44193    9.27574  1.0  0.0
 40.0  100.0   2.5095     9.24575  0.0  0.0
 41.0  100.0   2.563      9.22091  1.0  1.0
 42.0  100.0   2.09097    8.54326  1.0  1.0
 43.0  100.0   1.62172    7.8688   0.0  0.0
 44.0  100.0   1.69794    7.81702  1.0  0.0
 45.0  100.0   0.592244   8.23718  1.0  0.0
 46.0  100.0  -0.513572   8.65533  0.0  0.0
 47.0  10

### Fifth step
Write output to disk

In [7]:
## prep write to disk
using DelimitedFiles

## write output
writedlm( "out.tsv",  s, '\t')

## write landscape
writedlm( "landscape.tsv",  landscape, '\t')

# Sandbox